# コーラブル

## 関数

### 通常の関数

In [1]:
def cube(val)
	val ** 3
end

puts cube 6

216


* Rubyの関数を実行する際は,引数がない場合,単に関数名を示すだけで実行できる
	```Ruby
	def someFunc()
		puts "doSomething"
	end

	# 実行
	someFunc # doSomething
	```


	逆に,関数を値として渡すことはできない

* 関数内で外部の変数を参照するには,その変数が `$var` のようなグローバル変数でなければならない

### ラムダ (無名関数)

In [2]:
func = ->(val) {
	val ** 2
}

puts func

# 関数を利用
puts func.call(6)

#<Proc:0x00007ff370a01980 <main>:0 (lambda)>
36


* Rubyの無名関数は `Proc` オブジェクトであるが,これはブロックをオブジェクト化したものに過ぎない。  
	なので,上の式は次の式と実質的に違いがない
	```Ruby
	func = Proc.new do |val|
		val ** 2
	end
	```
	或いは,次の式とも違いがない
	```Ruby
	func = lambda { |val|
		val ** 2
	}
	```
	唯一の違いといえば,矢印を用いた式の方が引数の個数に対して厳格であることが挙げられる

* ブロックを受け取る関数は次のように実装できる  
	```Ruby
	# 実装
	def execBlock(arg1,arg2,&b)
		b.call(arg1,arg2)
	end

	# 利用
	execBlock(2,3) do |arg1,arg2|
		puts arg1
		puts arg2
	end
	```
	変数 `b` に対して `&b` としているので, `execBlock` に渡されたブロックは自動的に `Proc` オブジェクトになる  
	こうしたブロックを受け取る関数は,引数の最後でブロックを受け取る必要があり,2つ以上のブロックを引数で受け取ることはできない

	この関数は `yield` を使って次のようにも表せる。引数にブロックを明示する必要はなくなる
	```Ruby
	def execBlock(arg1,arg2)
		yield(arg1,arg2)
	end
	```

* デフォルト引数•キーワード引数

	次のように引数がデフォルト値を持った関数が作れる
	```Ruby
	def funcName(arg1="val1",arg2="val2",arg3="val3",...)
		...
    end
	```
	関数を実行する場合,デフォルト値を持つ引数は省略可能である
	しかし,この場合は特定の引数だけ与えることはできない (引数の順番は変えられない) ので,指定する引数までの他の引数も全て明示しないといけなくなる
    
	ここで, `Symbol` に似たキーワード引数を用いる
	```Ruby
	def funcName(arg1:"val1",arg2:"val2",arg3:"val3",...)
		...
    end
	```

	これを実行する際には特定の引数のみを指定することもできる
	```Ruby
	funcName(arg3:"customValue3",arg5:"customValue5")
	```

* 可変個引数

	残りの引数を配列形式でまとめて受け取れる
	```Ruby
	def func1(arg1,arg2,*args)
		...
		# args = [arg3,arg4...] : 配列
    end
	```

	キーワード引数をまとめてハッシュ形式で受け取ることができる
	```Ruby
	def func2(arg1,arg2,**args)
		...
		# args = {:arg3=>"val3",:arg4=>"val4"...} : ハッシュ
    end
	```

	又,関数を実行する際,配列の各要素をそのまま関数の引数とすることもできる
	```Ruby
	a = [1,2,3] # 配列aを用意
	func1(*a) # = func1(1,2,3) → arg1=1 arg2=2 args=[3]

	h = {"key1":1,"key2":2,"key3":3} # 連想配列hを用意
	func2(true,false,**h) # = func2(true,false,key1=1,key2=2,key3=3) → arg1=true arg2=false args={"key1":1,"key2":2,"key3":3}

	func1(*h) # = func1("key1","key2","key3") → arg1=["key1",1] arg2=["key2",2] args=[["key3",3]]
	```

## 文字列をRubyコードとして実行

### `eval(rubySource)` -> `value?`

* `rubySource` に記載したRubyコードを実行する
* ソース中の最終行の値が `eval` から返される
* 外の変数を `eval` 内で参照したり, `eval` 内の変数を外から参照することもできる